In [ ]:
#| default_exp core

# fastkaggle.core

> API details for fastkaggle.

In [ ]:
#|hide
from nbprocess.showdoc import *

In [ ]:
#|export
import os,json,re,importlib,pkg_resources
from fastcore.utils import *

In [ ]:
#|export
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [ ]:
#|export
def import_kaggle():
    "Import kaggle API, using Kaggle secrets `kaggle_username` and `kaggle_key` if needed"
    if iskaggle:
        from kaggle_secrets import UserSecretsClient
        sec = UserSecretsClient()
        os.environ['KAGGLE_USERNAME'] = sec.get_secret("kaggle_username")
        if not os.environ['KAGGLE_USERNAME']: raise Exception("Please insert your Kaggle username and key into Kaggle secrets")
        os.environ['KAGGLE_KEY'] = sec.get_secret("kaggle_key")
    from kaggle import api
    return api

In [ ]:
api = import_kaggle()
L(api.competitions_list())

NameError: name 'L' is not defined

In [ ]:
#|export
def split_pkgspec(pkgspec):
    "Split install string into list of triples (pkg, operator, targetver)"
    # e.g. "fastcore>=1.4.5" "fastai" ==> ('fastcore','>=','1.4.5') ('fastai','>','0')         
    pkg,*rest = re.split('(>=|<=|==|>|<)', pkgspec.replace('"', ''))
    try: operator,targetver = rest 
    except: operator,targetver = ">" , "0"
    return (pkg, operator, targetver)


In [ ]:
assert split_pkgspec('fastcore>=1.4.5') == ('fastcore', '>=', '1.4.5')
assert split_pkgspec('"fastcore>=1.4.5"') == ('fastcore', '>=', '1.4.5')
assert split_pkgspec('fastcore') == ('fastcore', '>', '0')
assert split_pkgspec('"fastcore"') == ('fastcore', '>', '0')
assert split_pkgspec('"fastcore==1.4.5"') == ('fastcore', '==', '1.4.5')

In [ ]:
#|export
def setup_install(install=''):
    "Set up python environment using pip-install to match string of pkgspecs"
    for pkgspec in install.replace('"', '').split():
        pkg, operator, targetver = split_pkgspec(pkgspec)

        try: oldver = pkg_resources.get_distribution(pkg).version
        except: pkg_resources.DistributionNotFound: oldver = '0'

        print('require', pkg, operator, targetver, '\n  found', oldver, end=',')
        if not eval(f"'{oldver}'{operator}'{targetver}'"): 
            print(' installing...', end='')
            os.system(f'pip install -Uqq "{pkgspec}"') 
            importlib.reload(pkg_resources)
            print( ' installed', pkg_resources.get_distribution(pkg).version, end=',')
               
        ver = __import__(pkg).__version__
        print(' active', ver)

In [ ]:
# Test downgrade one patch level
v1  = pkg_resources.get_distribution('fastcore').version
setup_install(install=f'"fastcore<{v1}""')
importlib.reload(pkg_resources)
v2 = pkg_resources.get_distribution('fastcore').version
assert pkg_resources.parse_version(v2) < pkg_resources.parse_version(v1)

# Test restore original version
setup_install(install=f'"fastcore=={v1}""')
importlib.reload(pkg_resources)
v3 = pkg_resources.get_distribution('fastcore').version
print(v3,v1)
assert pkg_resources.parse_version(v3) == pkg_resources.parse_version(v1)

require fastcore < 1.4.2 
  found 1.4.2, installing...

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbprocess 0.0.3 requires fastcore>=1.4.2, but you have fastcore 1.4.1 which is incompatible.


 installed 1.4.1, active 1.4.2
require fastcore > 0 
  found 1.4.1, active 1.4.2
1.4.1 1.4.2


AssertionError: 

In [ ]:
#|export
def setup_comp(competition, install=''):
    "Get a path to data for `competition`, downloading it if needed"
    setup_install(install)
 
    if iskaggle:
        return Path('../input')/competition
    else:
        path = Path(competition)
        from kaggle import api
        if not path.exists():
            import zipfile
            api.competition_download_cli(str(competition))
            zipfile.ZipFile(f'{competition}.zip').extractall(str(competition))
        return path

In [ ]:
setup_comp('titanic')

Path('titanic')

If you pass a list of space separated modules to `install`, they'll be installed if running on Kaggle.

In [ ]:
#|export
def nb_meta(user, id, title, file, competition=None, private=True, gpu=False, internet=True):
    "Get the `dict` required for a kernel-metadata.json file"
    d = {
      "id": f"{user}/{id}",
      "title": title,
      "code_file": file,
      "language": "python",
      "kernel_type": "notebook",
      "is_private": private,
      "enable_gpu": gpu,
      "enable_internet": internet,
      "keywords": [],
      "dataset_sources": [],
      "kernel_sources": []
    }
    if competition: d["competition_sources"] = [f"competitions/{competition}"]
    return d

In [ ]:
nb_meta('jhoward', 'my-notebook', 'My notebook', 'my-notebook.ipynb', competition='paddy-disease-classification')

{'id': 'jhoward/my-notebook',
 'title': 'My notebook',
 'code_file': 'my-notebook.ipynb',
 'language': 'python',
 'kernel_type': 'notebook',
 'is_private': True,
 'enable_gpu': False,
 'enable_internet': True,
 'keywords': [],
 'dataset_sources': [],
 'kernel_sources': [],
 'competition_sources': ['competitions/paddy-disease-classification']}

In [ ]:
#|export
def push_notebook(user, id, title, file, path='.', competition=None, private=True, gpu=False, internet=True):
    "Push notebook `file` to Kaggle Notebooks"
    meta = nb_meta(user, id, title, file=file, competition=competition, private=private, gpu=gpu, internet=internet)
    path = Path(path)
    nm = 'kernel-metadata.json'
    path.mkdir(exist_ok=True, parents=True)
    with open(path/nm, 'w') as f: json.dump(meta, f, indent=2)
    from kaggle import api
    api.kernels_push_cli(str(path))

Note that Kaggle recommends that the `id` match the *slug* for the title -- i.e it should be the same as the title, but lowercase, no punctuation, and spaces replaced with dashes. E.g:

```python
push_notebook('jhoward', 'first-steps-road-to-the-top-part-1',
              title='First Steps: Road to the Top, Part 1',
              file='first-steps-road-to-the-top-part-1.ipynb',
              competition='paddy-disease-classification',
              private=False, gpu=True)
```

## Export -

In [ ]:
#|hide
#|eval: false
from nbprocess.doclinks import nbprocess_export
nbprocess_export()